In [2]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 74.4 MB/s eta 0:00:00


In [4]:
# --- 0. Встановлення та Імпорти ---
!pip install gensim
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
import nltk
from nltk.corpus import twitter_samples, stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import re
import string

nltk.download('twitter_samples')
nltk.download('stopwords')

# --- 1. Завантаження векторів (FastText) ---
print(">>> Завантаження векторів слів (це може зайняти 1-2 хвилини)...")

# Англійська (Lang 1)
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gzip -d -f cc.en.300.vec.gz
# limit=50000 економить пам'ять. Для кращої якості можна збільшити до 100000, якщо Colab дозволяє
lang1_embeddings = KeyedVectors.load_word2vec_format("cc.en.300.vec", binary=False, limit=50000)

# Чеська (Lang 2)
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cs.300.vec.gz
!gzip -d -f cc.cs.300.vec.gz
lang2_embeddings = KeyedVectors.load_word2vec_format("cc.cs.300.vec", binary=False, limit=50000)

print(f"Завантажено EN слів: {len(lang1_embeddings)}")
print(f"Завантажено CS слів: {len(lang2_embeddings)}")

# --- 2. Завантаження словника MUSE (EN-CS) ---
print(">>> Завантаження словника перекладів...")
!wget -nc https://dl.fbaipublicfiles.com/arrival/dictionaries/en-cs.txt

def load_dict(filename):
    # Читаємо файл. Використовуємо on_bad_lines='skip' для нових версій pandas
    # Словники MUSE зазвичай розділені табуляцією (\t) або пробілом
    try:
        df = pd.read_csv(filename, sep='\t', header=None, names=['en', 'cs'], on_bad_lines='skip')
    except TypeError:
        # Фолбек для старих версій pandas
        df = pd.read_csv(filename, sep='\t', header=None, names=['en', 'cs'], error_bad_lines=False)

    # Якщо зчиталось в одну колонку, значить роздільник - пробіл
    if df.shape[1] < 2:
        try:
            df = pd.read_csv(filename, sep=' ', header=None, names=['en', 'cs'], on_bad_lines='skip')
        except TypeError:
            df = pd.read_csv(filename, sep=' ', header=None, names=['en', 'cs'], error_bad_lines=False)

    # Видаляємо пропуски та дублікати
    df = df.dropna().drop_duplicates()

    # Створюємо словник {англійське_слово: чеське_слово}
    return dict(zip(df['en'], df['cs']))

full_dict = load_dict('en-cs.txt')
print(f"Всього пар у словнику: {len(full_dict)}")

# Розбиваємо на Train (навчання) та Test (перевірка)
all_keys = list(full_dict.keys())
train_keys = all_keys[:5000]
test_keys = all_keys[5000:6500]

l1_l2_train = {k: full_dict[k] for k in train_keys if k in full_dict}
l1_l2_test = {k: full_dict[k] for k in test_keys if k in full_dict}

print(f"Розмір Train словника: {len(l1_l2_train)}")
print(f"Розмір Test словника: {len(l1_l2_test)}")

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


>>> Завантаження векторів слів (це може зайняти 1-2 хвилини)...
--2025-12-03 14:51:26--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.150.65, 108.157.150.95, 108.157.150.62, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.150.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G  49.7MB/s    in 2m 7s   

2025-12-03 14:53:35 (9.93 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]

--2025-12-03 14:55:01--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.cs.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.150.55, 108.157.150.95, 108.157.150.65, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.150.55|:443... connected.
HTTP request sent, awaiting respons

In [5]:
# --- 3. Формування матриць X та Y ---
def get_matrices(l1_l2, l2_embeddings, l1_embeddings):
    X_l = []
    Y_l = []
    for l1_word, l2_word in l1_l2.items():
        # Перевіряємо, чи є слова у завантажених векторах (FastText)
        if l1_word in l1_embeddings and l2_word in l2_embeddings:
            X_l.append(l1_embeddings[l1_word])
            Y_l.append(l2_embeddings[l2_word])
    return np.array(X_l), np.array(Y_l)

X_train, Y_train = get_matrices(l1_l2_train, lang2_embeddings, lang1_embeddings)
print(f"Розмірність матриць навчання: X={X_train.shape}, Y={Y_train.shape}")

# --- 4. Градієнтний спуск ---
def compute_loss(X, Y, R):
    m = X.shape[0]
    diff = np.dot(X, R) - Y
    loss = np.sum(diff**2) / m
    return loss

def compute_gradient(X, Y, R):
    m = X.shape[0]
    gradient = np.dot(X.T, np.dot(X, R) - Y) * (2 / m)
    return gradient

def align_embeddings(X, Y, train_steps=100, learning_rate=0.0003):
    np.random.seed(129)
    # Ініціалізація випадковою матрицею
    R = np.random.rand(X.shape[1], X.shape[1])

    for i in range(train_steps):
        if i % 100 == 0:
            print(f"Ітерація {i}: Loss = {compute_loss(X, Y, R):.4f}")
        gradient = compute_gradient(X, Y, R)
        R -= learning_rate * gradient
    return R

print(">>> Початок навчання матриці перетворення...")
# Параметри можна змінювати. learning_rate=0.8 добре підходить для нормалізованих векторів
R_train = align_embeddings(X_train, Y_train, train_steps=500, learning_rate=0.8)
print("Навчання завершено.")

# --- 5. Функція перекладу та Перевірка ---
def translate(word, R, l1_embeddings, l2_embeddings, k=1):
    if word not in l1_embeddings:
        return None
    # 1. Вектор слова
    v = l1_embeddings[word]
    # 2. Трансформація
    v_prime = np.dot(v, R)
    # 3. Пошук найближчого
    sims = l2_embeddings.cosine_similarities(v_prime, l2_embeddings.vectors)
    best_idxs = np.argsort(sims)[-k:][::-1]
    return [l2_embeddings.index_to_key[i] for i in best_idxs]

# Тест
test_word = "cat"
print(f"Переклад '{test_word}': {translate(test_word, R_train, lang1_embeddings, lang2_embeddings, k=3)}")

# --- 6. Точність (Accuracy) ---
def calculate_accuracy(l1_l2_test, R, l1_emb, l2_emb):
    correct = 0
    total = 0
    # Беремо підмножину для швидкості тестування
    test_items = list(l1_l2_test.items())[:500]

    for l1_word, l2_word in test_items:
        if l1_word in l1_emb and l2_word in l2_emb:
            total += 1
            pred = translate(l1_word, R, l1_emb, l2_emb, k=1)
            if pred and pred[0] == l2_word:
                correct += 1

    return correct / total if total > 0 else 0

acc = calculate_accuracy(l1_l2_test, R_train, lang1_embeddings, lang2_embeddings)
print(f"Точність на тестовій вибірці: {acc:.2%}")

Розмірність матриць навчання: X=(3717, 300), Y=(3717, 300)
>>> Початок навчання матриці перетворення...
Ітерація 0: Loss = 102.3514
Ітерація 100: Loss = 25.9041
Ітерація 200: Loss = 11.7877
Ітерація 300: Loss = 6.6362
Ітерація 400: Loss = 4.2729
Навчання завершено.
Переклад 'cat': ['pes', 'kocour', 'pejsek']
Точність на тестовій вибірці: 10.85%


In [6]:
# --- 7 & 8. Обробка твітів та Векторизація ---
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tokens = tokenizer.tokenize(tweet)

    return [stemmer.stem(w) for w in tokens if w not in stopwords_english and w not in string.punctuation]

def get_tweet_vector(tweet, model):
    tokens = process_tweet(tweet)
    vectors = [model[w] for w in tokens if w in model]
    if not vectors:
        return np.zeros(300)
    return np.mean(vectors, axis=0) # Усереднений вектор

# Завантажуємо твіти
all_tweets = twitter_samples.strings('positive_tweets.json') + twitter_samples.strings('negative_tweets.json')
# Беремо перші 2000 для швидкості демонстрації (можна взяти всі)
all_tweets = all_tweets[:2000]

print(">>> Векторизація твітів...")
tweet_vectors_dict = {i: get_tweet_vector(t, lang1_embeddings) for i, t in enumerate(all_tweets)}

# --- 9. LSH (Locality Sensitive Hashing) ---
def hash_func(vec, planes):
    # Скалярний добуток вектора на площини. Знак результату дає біт хешу.
    dot_product = np.dot(vec, planes)
    sign = (dot_product >= 0).astype(int)
    hash_val = 0
    for i, s in enumerate(sign):
        hash_val += s * (2**i)
    return hash_val

def make_hash_table(vecs_dict, planes):
    table = {}
    for idx, vec in vecs_dict.items():
        h = hash_func(vec, planes)
        table.setdefault(h, []).append(idx)
    return table

def create_lsh(vecs_dict, n_planes=10, n_tables=5):
    tables = []
    planes_list = []
    dim = list(vecs_dict.values())[0].shape[0]

    for _ in range(n_tables):
        planes = np.random.normal(size=(dim, n_planes))
        planes_list.append(planes)
        tables.append(make_hash_table(vecs_dict, planes))

    return tables, planes_list

print(">>> Створення LSH таблиць...")
tables, planes_list = create_lsh(tweet_vectors_dict, n_planes=10, n_tables=5)

# --- 10 & 11. Пошук сусідів ---
def lsh_knn(doc_id, vecs_dict, tables, planes_list, k=5):
    query_vec = vecs_dict[doc_id]
    candidates = set()

    # Збираємо кандидатів з усіх таблиць
    for table, planes in zip(tables, planes_list):
        h = hash_func(query_vec, planes)
        if h in table:
            candidates.update(table[h])

    if doc_id in candidates: candidates.remove(doc_id)

    # Сортуємо кандидатів за косинусною подібністю
    cand_list = list(candidates)
    if not cand_list: return []

    scores = []
    for cand_id in cand_list:
        cand_vec = vecs_dict[cand_id]
        cos_sim = np.dot(query_vec, cand_vec) / (np.linalg.norm(query_vec)*np.linalg.norm(cand_vec) + 1e-9)
        scores.append((cand_id, cos_sim))

    scores.sort(key=lambda x: x[1], reverse=True)
    return scores[:k]

# Тестування
query_id = 100 # Випадковий твіт
neighbors = lsh_knn(query_id, tweet_vectors_dict, tables, planes_list, k=3)

print(f"\nЗАПИТ:\n{all_tweets[query_id]}")
print("\nЗНАЙДЕНІ СХОЖІ:")
for n_id, score in neighbors:
    print(f"[Score {score:.2f}]: {all_tweets[n_id]}")

>>> Векторизація твітів...
>>> Створення LSH таблиць...

ЗАПИТ:
@metalgear_jp @Kojima_Hideo I want you're T-shirts ! They are so cool ! :D

ЗНАЙДЕНІ СХОЖІ:
[Score 0.70]: @virtuallykaren cool - thanks for thinking of me - hope all is well :-)
[Score 0.63]: It's still so nice out right now, I 💜 it! :) :) :)
[Score 0.57]: @IronMillTech Also, what is it you want me to do with this info? I shan't be ditching @ecotricity! And I'll die before I vote Tory...:)
